# Extra: Venue 7 electrics

If you see a straight line, it's interpolated missing data.  Try the sliders to see detail!  

This plot is interesting because it shows a minimum of around 2 kW base load with lots of 2.5 kW spikes even when the building is unoccupied.  If that electricity isn't doing useful work, getting those under control could substantially cut the electricity bill.   We aren't sure yet what it is, but it looks a bit like once before in another building when we found thermostatically controlled electric heating left on.  

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

df = pd.read_csv("venue-7-clampon-data.csv")
df["timestamp"] = pd.to_datetime(df['created_at'])
df = df.fillna(value=0)

phase1trace = go.Scatter(customdata=df, 
                    y=df['field1'], 
                    x = df['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='phase 1',
                    )

phase2trace = go.Scatter(customdata=df, 
                    y=df['field2'], 
                    x = df['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='phase 2',
                    )
phase3trace = go.Scatter(customdata=df, 
                    y=df['field3'], 
                    x = df['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='phase 3',
                    )

g = go.FigureWidget(data=[phase1trace,phase2trace,phase3trace])
g.layout.title = 'CurrentCost clamp-on meter readings'
g.layout.xaxis.title= 'timestamp'
g.layout.yaxis.title = "Watts"
g.layout.width = 1000
g.layout.height = 500

fig = go.Figure(g)

fig.update_layout(
    hovermode='x unified',
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)


# fig.update_yaxes(range=[50, 60])  



# fig.update_yaxes(range = [-5, df['temperature'].max()+5])

fig.show()

# second figure 

sumtrace = go.Scatter(customdata=df, 
                    y=df['field1'] + df['field2'] + df['field3'], 
                    x = df['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='sum of three phases',
                    )

g2 = go.FigureWidget(data=[sumtrace])
g2.layout.title = 'CurrentCost clamp-on meter readings - simple sum of phases'
g2.layout.xaxis.title= 'timestamp'
g2.layout.yaxis.title = "Watts"
g2.layout.width = 1000
g2.layout.height = 500

# print(g2)
# graph_objects does data validation, and for some reason on github but not my machine
# the figure has invalid stuff somewhere in the dictionary. Try skipping it.
fig2 = go.Figure(g2, skip_invalid="True")

#print(fig2)

fig2.update_layout(
    hovermode='x unified',
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

# Add range slider
fig2.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)


fig2.show()


We **think** this in watts, but the clamp-on meter is under-documented for three phase.  If it's not watts, the electricity use is a bit lower than what this says.   We still need to do the calculation for kilowatt-hours to sanity check this against some hand meter readings, although we know the metering has been OK in the past (on a different meter).

 The equipment we are using isn't very common and we only have one set, but getting a smart meter is another way of seeing this kind of information.  Admittedly, with half hourly readings a smart meter isn't nearly as informative.  This is roughly what the same data would look like with a smart meter:



In [ ]:


# downsample to every 30 minutes

df.field1 = df.field1.astype(int)
df.field2 = df.field2.astype(int) 
df.field3 = df.field3.astype(int)

# aggregate over 30 minute intervals - this is the interval for smart meter readings.
s = df.resample('30T', on='timestamp', origin='start').agg({'field1':'mean','field2':'mean','field3':'mean'})
s['timestamp'] = s.index 
s = s.fillna(value=0)

downsampled1 = go.Scatter(customdata=s, 
                    y=s['field1']/1000, 
                    x = s['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='phase 1',
                    )
downsampled2 = go.Scatter(customdata=s, 
                    y=s['field2']/1000, 
                    x = s['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='phase 2',
                    )
downsampled3 = go.Scatter(customdata=s, 
                    y=s['field3']/1000, 
                    x = s['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='phase 3',
                    )

g3 = go.FigureWidget(data=[downsampled1, downsampled2, downsampled3])
g3.layout.title = 'CurrentCost clamp-on meter readings - aggregated for half hour slots'
g3.layout.xaxis.title= 'timestamp'
g3.layout.yaxis.title = "kW"
g3.layout.width = 1000
g3.layout.height = 500

fig3 = go.Figure(g3)

fig3.update_layout(
    hovermode='x unified',
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

# Add range slider
fig3.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)


fig3.show()